In [1]:
#to open cred files
import os
#cleaning and operations
import pandas as pd
import numpy as np
#to read translations
import json
import unidecode
from urllib.parse import quote

In [2]:
from datetime import datetime

In [3]:
#to detect file types
import magic

In [4]:
#bigquery
from google.cloud import bigquery

In [5]:
#FIREBASE
from firebase_admin import credentials,storage,firestore,initialize_app

In [6]:
from dotenv import load_dotenv
load_dotenv('/Users/daniel/OAN/credentials/contoan/.env')

True

In [7]:
## OPEN CONECTION TO BIGQUERRY
bgq_client = bigquery.Client()
dataset_id = "{}.contoan".format(bgq_client.project)
dataset = bigquery.Dataset(dataset_id)

In [8]:
filename=os.environ['FIREBASE_FILENAME']
#OPEN CONECTION TO FIREBASE
cred = credentials.Certificate(filename)
firebase = initialize_app(cred)

db = firestore.client()

In [9]:
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

In [10]:
from oauth2client import file,client, tools
from googleapiclient import discovery
from httplib2 import Http

In [11]:
## CONECTION TO DRIVE

SCOPES = ['https://www.googleapis.com/auth/drive'
        ]
store = file.Storage('/Users/daniel/OAN/credentials/storage.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('/Users/daniel/OAN/credentials/credentials.json', SCOPES)
    creds = tools.run_flow(flow, store)
drive_service = discovery.build('drive', 'v3', http=creds.authorize(Http()))

In [12]:
sheet_service = discovery.build('sheets', 'v4', http=creds.authorize(Http()))

In [13]:
def upload_to_drive(file,filename,mimetype,place,folder_id):
    
    
    #metadatos del archivo
    file_metadata = {
        'name': filename,
        'parents': folder_id
    }
    media = MediaFileUpload(file,
                            mimetype=mimetype,
                            resumable=True)
    file = drive_service.files().create(body=file_metadata,
                                        media_body=media,
                                        supportsAllDrives=True,
                                        fields='id').execute()
    return file.get('id')

## Accounting table

### GET DATA

In [125]:
#get all the collection
col_query = db.collection('Accounting').stream()

In [126]:
#save it in a list of dictionaries
accounts = []
for acc in col_query:
    accounts.append(acc.to_dict())

### UPLOAD FILRES TO DRIVE

In [13]:
bucket = storage.bucket('oan-nikarit.appspot.com')

In [16]:
blob = bucket.get_blob('products/WhatsApp Video 2020-02-20 at 12.23.42.mp4')

datetime.datetime(2021, 2, 24, 0, 0)

In [20]:
blob.generate_signed_url(expiration=datetime(2021,2,24))

'https://storage.googleapis.com/oan-nikarit.appspot.com/products/WhatsApp%20Video%202020-02-20%20at%2012.23.42.mp4?Expires=1614124800&GoogleAccessId=firebase-adminsdk-p9lf1%40oan-nikarit.iam.gserviceaccount.com&Signature=UwzmLdtmHMMOnhl7zJd5gMePJ7%2BhyutXzj%2BKWNTlK1Apg%2BxT81PdFmtck7fgHv62up2kH2K8cHG8LEg0%2BONqanjUvnB1ULWQQKF2M%2BWG4aSKYG3vGCW4GFZJmGe0uK81UOhu3gcW0YYEDSVS7%2BhadZxFq5euHkJlGS5mugohYuFCqVhykOcVd4d1fkzgGPeVs%2FBOo%2FzxbE9vj%2BY57y2dBQHjUpb%2BHfXb%2BEFstAf8Fbpu6SpLnlRaCuSkmTQkhm2OperGcio8wX%2FodZN7Jx6%2BdeIyNy9ekUeWYH2iyy8Ki4fQ01it63ahPjNXuhEN554WQ3n%2BmDCQN%2F9zYFtp406N4g%3D%3D'

In [ ]:
#Carpeta de CONTOAN en drive
folder_id = {
    'spain':['1jVtC7HA_4LnMKfJCtDKNmXWhFnepI8Rd'],
    'benin':['1MwMSjForyIw_-ucg4_rCignPcBSmv6h0']
}

In [89]:
for account in accounts[2097:]:
    if not(account['images']):
        continue
    elif 0<len(account['images']):
        db_url = account['images'][0]['db_url']
        if not(db_url == None):
            doc_id = account['id']
            ##TITULO DEL ARCHIVO A SUBIR
            title=(account['execution_date'][0:4]+
                     account['execution_date'][5:7]+
                     account['execution_date'][8:10]+
                     '_'+doc_id)
            
            file_root = 'facturas/'+title
            #DESCARGAR EL ARCHIVO DE FIREBASE
            blob = bucket.get_blob(db_url)
            blob.download_to_filename(file_root)
            
            #detectar el mymetype (pdf,jpeg,png)
            mimetype = magic.from_file(file_root,mime=True)
            ##COGERLO
            content_type = blob.content_type
            
            filename=title+mimetype.split('/')[1]
            
            #subir a drive
            drive_id = upload_to_drive(file_root,
                                       filename,
                                       mimetype,folder_id[account['place']])
            
            ##actualizar documento en FIREBASE
            
            account['images'][0]['drive_id']=drive_id
            account['images'][0]['drive_url']="https://drive.google.com/file/d/"+drive_id+"/view"
            new_images = account['images']
            db.collection(u'Accounting').document(doc_id).update({'images':new_images})

## SOLVE BIG CLEANING OF DB

In [12]:
files_spain = []
results = drive_service.files().list(q="'1jVtC7HA_4LnMKfJCtDKNmXWhFnepI8Rd' in parents",
                                     corpora='drive',
                                     pageSize=500,
                                     driveId = "0AMTa4DNuUN_NUk9PVA",
                                     fields="nextPageToken, files(id, name)",
                                     includeItemsFromAllDrives=True,
                                     supportsAllDrives=True).execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    for item in items:
        files_spain.append({"name":item['name'],"drive_id":item['id']})

In [13]:
files_nikki = []
results = drive_service.files().list(q="'1MwMSjForyIw_-ucg4_rCignPcBSmv6h0' in parents",
                                     corpora='drive',
                                     pageSize=500,
                                     driveId = "0AMTa4DNuUN_NUk9PVA",
                                     fields="nextPageToken, files(id, name)",
                                     includeItemsFromAllDrives=True,
                                     supportsAllDrives=True).execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    for item in items:
        files_nikki.append({"name":item['name'],"drive_id":item['id']})

In [14]:
files_drive = files_nikki+files_spain

In [15]:
len(files_nikki)

110

In [16]:
len(files_spain)

101

In [17]:
len(files_drive)

211

In [18]:
bucket = storage.bucket('oan-nikarit.appspot.com')

In [19]:
blobs = []
for blob in bucket.list_blobs():
    blobs.append(blob)

In [460]:
o = blobs[0]

In [461]:
o.metadata

{'firebaseStorageDownloadTokens': '53c1c740-3af5-4efc-8d1c-1668bbf2acce'}

In [20]:
len(blobs)

344

In [456]:
326-len(files_drive)

115

In [21]:
ini="https://firebasestorage.googleapis.com/v0/b/"
nex="?alt=media&token="
b_name = bucket.name

In [164]:
l_blobs = [{'name':b.name,
  'download_url': ini+b_name+ "/o/" + quote(b.name,safe='~()*!.\'')+nex+b.metadata["firebaseStorageDownloadTokens"],
  'uuid':b.metadata["firebaseStorageDownloadTokens"],
  'creation_date':b.time_created.isoformat(),
  'id_blob':b.id,
  'folder':b.name.split('/')[0],
  'storage_class':b.storage_class,
  'size':b.size,
  'content_type':b.content_type
  } for b in blobs]

In [165]:
df_blobs = pd.DataFrame(l_blobs)

In [24]:
df_blobs['content_type'].value_counts()

image/jpeg         220
application/pdf    101
image/png           23
Name: content_type, dtype: int64

In [25]:
df_blobs['folder'].value_counts()

accounting    326
users          10
products        8
Name: folder, dtype: int64

In [268]:
df_blobs

,name,download_url,uuid,creation_date,id_blob,folder,storage_class,size,content_type
0,accounting/20201010104530,https://firebasestorage.googleapis.com/v0/b/oa...,53c1c740-3af5-4efc-8d1c-1668bbf2acce,2020-11-10T09:45:32.909000+00:00,oan-nikarit.appspot.com/accounting/20201010104...,accounting,STANDARD,306536,application/pdf
1,accounting/20201010105833,https://firebasestorage.googleapis.com/v0/b/oa...,682fa416-5dd5-4f31-8006-faf0831ba499,2020-11-10T09:58:35.845000+00:00,oan-nikarit.appspot.com/accounting/20201010105...,accounting,STANDARD,6572933,image/jpeg
2,accounting/20201010105859,https://firebasestorage.googleapis.com/v0/b/oa...,5528bde0-d07c-4e74-b0de-042cbee65a75,2020-11-10T09:59:01.441000+00:00,oan-nikarit.appspot.com/accounting/20201010105...,accounting,STANDARD,5927892,image/jpeg
3,accounting/2020101011131,https://firebasestorage.googleapis.com/v0/b/oa...,79397969-c1b5-4313-8108-989139d1627b,2020-11-10T10:13:04.034000+00:00,oan-nikarit.appspot.com/accounting/20201010111...,accounting,STANDARD,5680962,image/jpeg
4,accounting/20201017164233,https://firebasestorage.googleapis.com/v0/b/oa...,ef86a7b6-dea3-41c3-8314-42d5744039e5,2020-11-17T15:42:51.826000+00:00,oan-nikarit.appspot.com/accounting/20201017164...,accounting,STANDARD,4313048,image/jpeg
...,...,...,...,...,...,...,...,...,...
339,users/202042415524,https://firebasestorage.googleapis.com/v0/b/oa...,bde165a5-f981-4b31-879c-49c02d524d29,2020-05-24T13:05:26.813000+00:00,oan-nikarit.appspot.com/users/202042415524/159...,users,STANDARD,603018,image/png
340,users/202048171826,https://firebasestorage.googleapis.com/v0/b/oa...,80e1bd99-dabc-452c-8410-11cf06b1b75d,2020-05-08T15:18:27.402000+00:00,oan-nikarit.appspot.com/users/202048171826/158...,users,STANDARD,133665,image/jpeg
341,users/20204817198,https://firebasestorage.googleapis.com/v0/b/oa...,65830e45-68c6-43af-bb1a-977b4f6e5fd8,2020-05-08T15:19:08.822000+00:00,oan-nikarit.appspot.com/users/20204817198/1588...,users,STANDARD,128825,image/jpeg
342,users/2020828112911,https://firebasestorage.googleapis.com/v0/b/oa...,bc312e96-4659-4068-9b43-4beb0a82c0ea,2020-09-28T10:29:36.671000+00:00,oan-nikarit.appspot.com/users/2020828112911/16...,users,STANDARD,4373480,image/jpeg


In [284]:
df_blobs_gbq = pd.merge(df_blobs,accounting[['id','drive_url','download_url']],on='download_url',how='left')

#### NOT MATCH

In [336]:
not_match = df_blobs_gbq[(df_blobs_gbq['folder']=='accounting') &
             (df_blobs_gbq['id'].isna())]

In [337]:
not_match_names = df_blobs_gbq[(df_blobs_gbq['folder']=='accounting') &
             (df_blobs_gbq['id'].isna())]['name'].to_list()

In [457]:
not_match.shape

(114, 11)

### ONE TO ONE

In [393]:
not_match["2021-01-08" < not_match['creation_date'].apply(str).str[0:10]]

,name,download_url,uuid,creation_date,id_blob,folder,storage_class,size,content_type,id,drive_url
325,accounting/20210920292820203112_etiqueta_prods,https://firebasestorage.googleapis.com/v0/b/oa...,6d2d0595-3695-4610-b51e-4c8698968404,2021-01-09T19:29:29.154000+00:00,oan-nikarit.appspot.com/accounting/20210920292...,accounting,STANDARD,185269,application/pdf,NaN,NaN


In [394]:
db.collection('Accounting').document('RW3jGJJfia811blYOBVW').get().to_dict()

{'code': 2577,
 'images': [{'file': {},
   'drive_id': '11OKED_pOgH_GRj8HX__dPnyTyeJ5PblF',
   'drive_url': 'https://drive.google.com/file/d/1YoG52-30QSgi7GIdO0ZuTAMP2CAOP5RE/view',
   'db_url': nan,
   'download_url': nan,
   'img_url': ''}],
 'concept': 'Etiquetas de manteca edición navidad',
 'origin': 'contoan',
 'place': 'spain',
 'id': 'RW3jGJJfia811blYOBVW',
 'type': 'expense',
 'execution_date': '2020-12-31T20:22:45.982+01:00',
 'project': '0DmODGTOEiM5lg9SGx0J',
 'creation_date': '2021-01-09T19:22:45.982Z',
 'user_in_charge': 'exFnSrTD3fykjKbfaAvt',
 'intervention': 'w04441aFcU5b7pQm6Rd2',
 'amount': 26.62,
 'description': '50 etiquetas',
 'vat_amount': 4.619999999999999,
 'account_id': 'WVxyN2FodiVUq7uDkEmc',
 'creator_user': 'z5m936GA0t3vHM28QKhR',
 'target_id': 'vJbbj1kPxkcdXJyBOf1l',
 'phase': 'gtjyWLZn07xtj3ZdZcZ5',
 'vat': 21}

In [395]:
not_match.shape

(114, 11)

In [385]:
not_match[:-4]

,name,download_url,uuid,creation_date,id_blob,folder,storage_class,size,content_type,id,drive_url
2,accounting/20201010105859,https://firebasestorage.googleapis.com/v0/b/oa...,5528bde0-d07c-4e74-b0de-042cbee65a75,2020-11-10T09:59:01.441000+00:00,oan-nikarit.appspot.com/accounting/20201010105...,accounting,STANDARD,5927892,image/jpeg,NaN,NaN
3,accounting/2020101011131,https://firebasestorage.googleapis.com/v0/b/oa...,79397969-c1b5-4313-8108-989139d1627b,2020-11-10T10:13:04.034000+00:00,oan-nikarit.appspot.com/accounting/20201010111...,accounting,STANDARD,5680962,image/jpeg,NaN,NaN
6,accounting/20201017165322,https://firebasestorage.googleapis.com/v0/b/oa...,cf3ffa22-87cc-44a4-89b3-1cbddf045f24,2020-11-17T15:53:45.648000+00:00,oan-nikarit.appspot.com/accounting/20201017165...,accounting,STANDARD,2922767,image/jpeg,NaN,NaN
7,accounting/20201017165818,https://firebasestorage.googleapis.com/v0/b/oa...,f54a768e-9ead-42c8-ac7b-4b7ec49974f7,2020-11-17T15:58:46.110000+00:00,oan-nikarit.appspot.com/accounting/20201017165...,accounting,STANDARD,2922767,image/jpeg,NaN,NaN
9,accounting/20201017171729,https://firebasestorage.googleapis.com/v0/b/oa...,f1ed7ee0-d898-451d-8050-1d4c9166559d,2020-11-17T16:18:12.502000+00:00,oan-nikarit.appspot.com/accounting/20201017171...,accounting,STANDARD,4946104,image/jpeg,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
280,accounting/20208714317,https://firebasestorage.googleapis.com/v0/b/oa...,5e25e92e-a0a2-4a3e-80fc-e611f34c9564,2020-09-07T12:03:18.641000+00:00,oan-nikarit.appspot.com/accounting/20208714317...,accounting,STANDARD,115390,image/png,NaN,NaN
290,accounting/2020911421,https://firebasestorage.googleapis.com/v0/b/oa...,d58b93fd-81c9-40d9-9b98-c6764a1ece4f,2020-09-30T23:04:23.358000+00:00,oan-nikarit.appspot.com/accounting/2020911421/...,accounting,STANDARD,119466,application/pdf,NaN,NaN
292,accounting/202091513349,https://firebasestorage.googleapis.com/v0/b/oa...,5fb189fe-f9c8-407a-b4fb-012e5b566657,2020-10-14T23:33:51.812000+00:00,oan-nikarit.appspot.com/accounting/20209151334...,accounting,STANDARD,4002789,image/jpeg,NaN,NaN
293,accounting/202091513710,https://firebasestorage.googleapis.com/v0/b/oa...,9ce83fd2-558a-4f78-a2a2-a3cd212b870f,2020-10-14T23:37:11.036000+00:00,oan-nikarit.appspot.com/accounting/20209151371...,accounting,STANDARD,152749,image/png,NaN,NaN


In [389]:
c1 = not_match.loc[293]
c1

name                                       accounting/202091513710
download_url     https://firebasestorage.googleapis.com/v0/b/oa...
uuid                          9ce83fd2-558a-4f78-a2a2-a3cd212b870f
creation_date                     2020-10-14T23:37:11.036000+00:00
id_blob          oan-nikarit.appspot.com/accounting/20209151371...
folder                                                  accounting
storage_class                                             STANDARD
size                                                        152749
content_type                                             image/png
id                                                             NaN
drive_url                                                      NaN
Name: 293, dtype: object

In [390]:
c1['download_url']

'https://firebasestorage.googleapis.com/v0/b/oan-nikarit.appspot.com/o/accounting%2F202091513710?alt=media&token=9ce83fd2-558a-4f78-a2a2-a3cd212b870f'

In [391]:
c1['name']

'accounting/202091513710'

In [392]:
accounting[accounting['creation_date'].apply(str).str[0:10] == c1['creation_date'][0:10]]

,images,creation_date,user_in_charge,code,account_id,vat_amount,intervention,project,id,concept,...,vat,execution_date,description,origin,type,place,creator_user,receipt,drive_url,download_url


### MATCH

In [376]:
c1_match = accounting[accounting['id']=='MjXD3X8YjpMSObftp5Su']

In [377]:
c1_match['concept']

858    Gastos de Envío Geopost  SEUR GEOPOST AGOSTO
Name: concept, dtype: object

In [378]:
c1_match['images'].iloc[0]

[{'db_url': 'accounting/202101910416',
  'file': {},
  'download_url': 'https://firebasestorage.googleapis.com/v0/b/oan-nikarit.appspot.com/o/accounting%2F202101910416?alt=media&token=8c8f8aa0-bb9f-49ff-aa50-aca8631f0e2b',
  'img_url': ''}]

In [ ]:
not_match_ids = {
    'accounting/2021062258120202103_fbads':'jMuJTXlvctWHjzetRyXz',
    'accounting/20210920292820203112_etiqueta_prods':'RW3jGJJfia811blYOBVW',
    'accounting/202108222549African Sky Cargo':'F0vfoEOfTATgJRKpqqKd',
    'accounting/202097215220':None,#'w2zAUVhhRq59cxY87IDj' - esta repetida aqui
    'accounting/20209158460':None,#'MjXD3X8YjpMSObftp5Su' - esta repetida -296
}

name                          accounting/2021062258120202103_fbads
download_url     https://firebasestorage.googleapis.com/v0/b/oa...
uuid                          c5380fbf-a864-460c-8e27-742facb52baa
creation_date                     2021-01-06T21:58:02.818000+00:00
id_blob          oan-nikarit.appspot.com/accounting/20210622581...
folder                                                  accounting
storage_class                                             STANDARD
size                                                       1037347
content_type                                       application/pdf
id                                            jMuJTXlvctWHjzetRyXz
drive_url                                                      NaN
Name: 323, dtype: object

In [396]:
table_id = "{}.files".format(dataset.dataset_id)
proj = bgq_client.project
df_blobs_gbq.to_gbq(table_id,proj,if_exists='replace')

1it [00:05,  5.06s/it]


In [458]:
df_blobs_gbq

,name,download_url,uuid,creation_date,id_blob,folder,storage_class,size,content_type,id,drive_url
0,accounting/20201010104530,https://firebasestorage.googleapis.com/v0/b/oa...,53c1c740-3af5-4efc-8d1c-1668bbf2acce,2020-11-10T09:45:32.909000+00:00,oan-nikarit.appspot.com/accounting/20201010104...,accounting,STANDARD,306536,application/pdf,M2KRJszoSiwo50ZfuKVa,https://drive.google.com/file/d/1AtcofvKA03ttA...
1,accounting/20201010105833,https://firebasestorage.googleapis.com/v0/b/oa...,682fa416-5dd5-4f31-8006-faf0831ba499,2020-11-10T09:58:35.845000+00:00,oan-nikarit.appspot.com/accounting/20201010105...,accounting,STANDARD,6572933,image/jpeg,evfSU4JBTOE9yFLkM9H8,https://drive.google.com/file/d/1Unie9GXAxBN0p...
2,accounting/20201010105859,https://firebasestorage.googleapis.com/v0/b/oa...,5528bde0-d07c-4e74-b0de-042cbee65a75,2020-11-10T09:59:01.441000+00:00,oan-nikarit.appspot.com/accounting/20201010105...,accounting,STANDARD,5927892,image/jpeg,NaN,NaN
3,accounting/2020101011131,https://firebasestorage.googleapis.com/v0/b/oa...,79397969-c1b5-4313-8108-989139d1627b,2020-11-10T10:13:04.034000+00:00,oan-nikarit.appspot.com/accounting/20201010111...,accounting,STANDARD,5680962,image/jpeg,NaN,NaN
4,accounting/20201017164233,https://firebasestorage.googleapis.com/v0/b/oa...,ef86a7b6-dea3-41c3-8314-42d5744039e5,2020-11-17T15:42:51.826000+00:00,oan-nikarit.appspot.com/accounting/20201017164...,accounting,STANDARD,4313048,image/jpeg,WDevSAQwaFPwW8pzDYHx,https://drive.google.com/file/d/1fz9yCXtS9g7iM...
...,...,...,...,...,...,...,...,...,...,...,...
339,users/202042415524,https://firebasestorage.googleapis.com/v0/b/oa...,bde165a5-f981-4b31-879c-49c02d524d29,2020-05-24T13:05:26.813000+00:00,oan-nikarit.appspot.com/users/202042415524/159...,users,STANDARD,603018,image/png,NaN,NaN
340,users/202048171826,https://firebasestorage.googleapis.com/v0/b/oa...,80e1bd99-dabc-452c-8410-11cf06b1b75d,2020-05-08T15:18:27.402000+00:00,oan-nikarit.appspot.com/users/202048171826/158...,users,STANDARD,133665,image/jpeg,NaN,NaN
341,users/20204817198,https://firebasestorage.googleapis.com/v0/b/oa...,65830e45-68c6-43af-bb1a-977b4f6e5fd8,2020-05-08T15:19:08.822000+00:00,oan-nikarit.appspot.com/users/20204817198/1588...,users,STANDARD,128825,image/jpeg,NaN,NaN
342,users/2020828112911,https://firebasestorage.googleapis.com/v0/b/oa...,bc312e96-4659-4068-9b43-4beb0a82c0ea,2020-09-28T10:29:36.671000+00:00,oan-nikarit.appspot.com/users/2020828112911/16...,users,STANDARD,4373480,image/jpeg,NaN,NaN


### BACKUP

In [28]:
sheet = sheet_service.spreadsheets()

In [31]:
SPREADSHEET_ID = "10me8N68cAVXVBXYU_sTub5F_7pKsDtTT29giJ0FGAzw"

In [ ]:
values = result.get('values', []).

In [33]:
rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range = "TOUT").execute()

In [39]:
backup = pd.DataFrame(rows['values'][1:],columns=rows['values'][0])

In [42]:
dtcols = ['CREATION','DATE EXECUTION']
numcols = ['QUANTITÉ']

In [43]:
for col in dtcols:
    backup[col] = pd.to_datetime(backup[col])

In [46]:
#for col in numcols:
#    backup[col] = backup[col].apply(lambda x: float(x.replace('CFA','')))

In [40]:
print(backup.columns)

Index(['ID', 'CREATION', 'DATE EXECUTION', 'CONCEPT', 'PROJECT',
       'INTERVENTION', 'PHASE', 'TYPE', 'QUANTITÉ', 'CAISEE / BANQUE',
       'CATEGORIE / BANQUE', 'DESCRIPTION', 'CREATEUR', 'RESPONSABLE', 'LIEU',
       'FACTURE'],
      dtype='object')


In [50]:
max_backup = backup['CREATION'].max()

In [48]:
df_accounting = pd.read_gbq('SELECT * FROM `oan-nikarit.contoan.accounting`')

Downloading: 100%|██████████| 2400/2400 [00:01<00:00, 1459.34rows/s]


In [62]:
df_accounting.shape

(2400, 20)

In [66]:
str(max_backup.tz_localize('UTC'))

'2021-08-01 00:00:00+00:00'

In [61]:
df_accounting['creation_date'].max()

Timestamp('2021-01-21 14:17:58.103000+0000', tz='UTC')

In [91]:
df_accounting_bu = df_accounting[df_accounting['creation_date'].apply(str).str[0:10] < "2021-01-08"].copy()

In [93]:
len(df_accounting_bu['receipt'].unique())

334

In [94]:
len(backup['FACTURE'].unique())

291

In [95]:
col_query = db.collection('Accounting').stream()
accounts = []
for acc in col_query:
    accounts.append(acc.to_dict())

In [96]:
df_accounts = pd.DataFrame(accounts)

In [129]:
df_accounts_notna = df_accounts[~(df_accounts['images'].isna())]

In [135]:
df_accounts_with_r = df_accounts_notna[~df_accounts_notna['images'].apply(lambda x: x ==[])]

In [397]:
df_accounts_with_r['images'].shape

(357,)

### LET SOLVE THE PROBLEM

In [398]:
df_accounts_with_r['images'].iloc[17]

[{'drive_id': '1Po_mVED26HYAtHzQ4GOtYauftLTDh5Je',
  'title': '20201221_fbads',
  'download_url': 'https://firebasestorage.googleapis.com/v0/b/oan-nikarit.appspot.com/o/accounting%2F20210622282620201221_fbads?alt=media&token=b9f0106f-9baa-4502-bea4-d38203ddba61',
  'file': {},
  'img_url': '',
  'db_url': 'accounting/20210622282620201221_fbads',
  'drive_url': 'https://drive.google.com/file/d/1Po_mVED26HYAtHzQ4GOtYauftLTDh5Je/view'}]

### WITH RECEIPT

In [399]:
df_accounts_with_r = df_accounts_with_r.copy()

In [432]:
df_accounts_with_r.shape

(357, 22)

In [438]:
def im_search(x):
    download_url = ''
    drive_url = ''
    if (0 < len(x)):
        im = x[0]
        if 'download_url' in im.keys():
            download_url = im['download_url']
        if 'drive_url' in im.keys():
            drive_url = im['drive_url'] 
    return pd.Series([drive_url,download_url])

In [439]:
df_accounts_with_r[['drive_url','download_url']] = df_accounts_with_r.images.apply(lambda x: im_search(x))

In [443]:
df_accounts_with_r[(df_accounts_with_r['download_url'].isna()) |
                   (df_accounts_with_r['download_url']=='')].shape

(25, 22)

In [444]:
df_accounts_backup_2 = pd.merge(df_accounts_with_r,backup[['ID','FACTURE']],
                                left_on = 'id',right_on = 'ID',how='left')

In [445]:
df_accounts_backup_2['fire_update'] = df_accounts_backup_2['download_url'].combine_first(
                                                                df_accounts_backup_2['FACTURE'])

In [446]:
df_accounts_backup_2[
    ~(df_accounts_backup_2['download_url'].isna()) &
    ~(df_accounts_backup_2['FACTURE'].isna()) &
    (df_accounts_backup_2['FACTURE'] != df_accounts_backup_2['download_url'])
][['fire_update','download_url','FACTURE']]

,fire_update,download_url,FACTURE
9,https://firebasestorage.googleapis.com/v0/b/oa...,https://firebasestorage.googleapis.com/v0/b/oa...,https://firebasestorage.googleapis.com/v0/b/oa...
123,https://firebasestorage.googleapis.com/v0/b/oa...,https://firebasestorage.googleapis.com/v0/b/oa...,https://firebasestorage.googleapis.com/v0/b/oa...


In [448]:
df_accounts_backup_2[(df_accounts_backup_2['download_url'].isna()) |
                   (df_accounts_backup_2['download_url']=='')]['FACTURE'].value_counts()

Series([], Name: FACTURE, dtype: int64)

### WITHOUT RECEIPT

In [149]:
df_accounts_nr = df_accounts[~df_accounts.index.isin(df_accounts_with_r.index)].copy()

In [207]:
df_accounts_nr.shape

(2231, 21)

In [206]:
df_accounts_nr['images'].value_counts()

[]    2226
Name: images, dtype: int64

In [172]:
df_accounts_nr = df_accounts_nr.reset_index()

In [173]:
df_accounts_nr.shape

(2231, 21)

In [174]:
df_accounts_backup = pd.merge(df_accounts_nr['id'],backup[['ID','FACTURE']],left_on = 'id',right_on = 'ID',how='left')

In [175]:
df_accounts_backup = df_accounts_backup[['id','FACTURE']].copy()

In [176]:
df_accounts_backup.columns = ['id','download_url']

In [177]:
df_accounts_bu_blob = pd.merge(df_accounts_backup,df_blobs,on='download_url',how='left')

In [178]:
df_accounts_bu_blob = df_accounts_bu_blob[['id','download_url','name']].copy()b

In [192]:
def search_drive(x):
    d_url = None
    d_name = None
    for fil in files_drive:
        f_id = fil['name'].split('_')[1].split('.')[0]      
        if f_id == x:
            d_url = "https://drive.google.com/file/d/"+fil['drive_id']+"/view"
            d_name = fil['name']
    return pd.Series([d_url,d_name,fil['drive_id']])

In [193]:
df_accounts_bu_blob[['drive_url','drive_name','drive_id']] = df_accounts_bu_blob['id'].apply(lambda x: search_drive(x))

In [194]:
df_accounts_recup = df_accounts_bu_blob[~(df_accounts_bu_blob['download_url'].isna()) |
                        ~(df_accounts_bu_blob['drive_url'].isna()) |
                        ~(df_accounts_bu_blob['drive_name'].isna()) |
                        ~(df_accounts_bu_blob['name'].isna())].copy()

In [195]:
df_accounts_recup = df_accounts_recup.reset_index()

## UPLOAD TO FIREBASE

In [228]:
recup = df_accounts_recup.T.to_dict()

for r in recup:
    row= recup[r]
    doc_id = row['id']
    im = [{'drive_id': row['drive_id'],
      'db_url': row['name'],
      'file': {},
      'img_url': '',
      'download_url': row['download_url'],
      'drive_url': row['drive_url']}]
    #db.collection('Accounting').document(doc_id).update({'images':im})

### CHECK

In [229]:
col_query = db.collection('Accounting').stream()
accounts = []
for acc in col_query:
    accounts.append(acc.to_dict())

In [230]:
df_accounts = pd.DataFrame(accounts)

In [231]:
df_accounts_notna = df_accounts[~(df_accounts['images'].isna())]

In [232]:
df_accounts_with_r = df_accounts_notna[~df_accounts_notna['images'].apply(lambda x: x ==[])]

In [233]:
df_accounts_with_r['images'].shape

(357,)

## EDA

In [234]:
#convert to dataframe
accounting = pd.DataFrame(accounts)

In [235]:
#errores_ids = accounting[(accounting['amount'].apply(lambda x: type(x) == type('2')))]['id'].tolist()
#for error in errores_ids:
#    value = float(accounting[accounting['id']==error]['amount'])
#    db.collection('Accounting'
#                 ).document(error
#                           ).update({'amount':value}
#                                   )

In [236]:
#nulls. size and datatype
accounting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2403 entries, 0 to 2402
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   images          2398 non-null   object 
 1   creation_date   2403 non-null   object 
 2   user_in_charge  2403 non-null   object 
 3   code            2403 non-null   int64  
 4   account_id      2403 non-null   object 
 5   vat_amount      2384 non-null   float64
 6   intervention    2403 non-null   object 
 7   project         2403 non-null   object 
 8   id              2403 non-null   object 
 9   concept         2403 non-null   object 
 10  target_id       2403 non-null   object 
 11  phase           800 non-null    object 
 12  amount          2403 non-null   float64
 13  vat             2165 non-null   float64
 14  execution_date  2403 non-null   object 
 15  description     2350 non-null   object 
 16  origin          2403 non-null   object 
 17  type            2403 non-null   o

In [237]:
#nulls heatmap - import seaborn as sns
#sns.heatmap(accounting.isna())

### CLEAN DATA

In [238]:
#lets see how it shows
accounting.head()

,images,creation_date,user_in_charge,code,account_id,vat_amount,intervention,project,id,concept,target_id,phase,amount,vat,execution_date,description,origin,type,place,creator_user
0,[],2020-08-26T18:02:55.591Z,IjBxXuBshlfq2MUzwBSK,899,3deqBmrZ1iJkAwMmxSOp,3.644628,w04441aFcU5b7pQm6Rd2,0DmODGTOEiM5lg9SGx0J,01WVDHF0gSSg1PCFsosk,venta shopify,yfy4cPxkmFwIYVnoWD0A,None,21.0,21.0,2020-04-28T12:07:49.000Z,venta online: 2288996286563,script_dani,income,spain,z5m936GA0t3vHM28QKhR
1,[],2020-12-22T08:59:35.685Z,IjBxXuBshlfq2MUzwBSK,2257,FHwFsBItyEneaZh8yP0T,0.000000,w04441aFcU5b7pQm6Rd2,0DmODGTOEiM5lg9SGx0J,01dtZiGq12IzmIpOVG3X,Gastos de envio Nacional,iA9Pzv2CImjItzwCaQv0,None,6.1,0.0,2020-12-22T08:59:35.685Z,2915979624547,shopify,movement,spain,z5m936GA0t3vHM28QKhR
2,[],2020-12-07T08:56:28.137Z,IjBxXuBshlfq2MUzwBSK,1959,FHwFsBItyEneaZh8yP0T,0.000000,w04441aFcU5b7pQm6Rd2,0DmODGTOEiM5lg9SGx0J,01nQGx25hZLez38FpWEe,Gastos de envio Nacional,iA9Pzv2CImjItzwCaQv0,None,6.1,0.0,2020-12-07T08:56:28.137Z,2875805401187,shopify,movement,spain,z5m936GA0t3vHM28QKhR
3,[],2020-08-26T18:02:55.591Z,IjBxXuBshlfq2MUzwBSK,1049,3deqBmrZ1iJkAwMmxSOp,6.595041,w04441aFcU5b7pQm6Rd2,0DmODGTOEiM5lg9SGx0J,026RXV3hyl33ussoU7D9,venta shopify,iA9Pzv2CImjItzwCaQv0,None,38.0,21.0,2020-04-28T19:50:24.000Z,venta online: 2290422218851,script_dani,income,spain,z5m936GA0t3vHM28QKhR
4,[],2020-12-05T22:24:00.338Z,IjBxXuBshlfq2MUzwBSK,1934,j2UYEKf01eFHMPZv9qHq,0.000000,w04441aFcU5b7pQm6Rd2,0DmODGTOEiM5lg9SGx0J,02EE48NlGSbqGrrLvIHl,Comision Stripe,iA9Pzv2CImjItzwCaQv0,LXspD4R7rZ6Jw5EOEZYR,1.2,0.0,2020-12-05T22:24:00.338Z,2872447467619,shopify,expense,spain,z5m936GA0t3vHM28QKhR


#### convert execution to isostring

In [239]:
def clean_execution(x):
    if(len(x['execution_date'])<15):
        doc_id = x['id']
        exec_date=x['execution_date']+'T14:14:58.593Z'
        print(exec_date)
        print(len(x['execution_date']))
        #upload to firebase if whished
        #a=db.collection(u'Accounting').document(doc_id).update({'execution_date':exec_date})
        
        return exec_date
    else:
        return x['execution_date']
        

In [240]:
accounting['execution_date']=accounting.apply(lambda x: clean_execution(x) ,axis=1)

2021-11-30T14:14:58.593Z
10


##### clean VAT and replace nan with 0

In [241]:
def clean_vat_amount(x):
    doc_id = x['id']
    amount=x['amount']
    origin = x['origin']
    if origin =='script_dani':
        if 0 < x['vat']:
            vat=21
        else:
            vat=0
        #a=db.collection(u'Accounting').document(doc_id).update({'vat':vat})
    else:
        vat=x['vat']
        if not(0 <= vat):
            vat=0
    vat_amount=amount*(1-1/(1+vat/100))

    #a=db.collection(u'Accounting').document(doc_id).update({'vat_amount':vat_amount})

    return pd.Series({'vat':vat,'vat_amount':vat_amount})

In [242]:
accounting[['vat','vat_amount']] = accounting.apply(lambda x: clean_vat_amount(x),axis=1)

###### convert dates to datetime

In [243]:
#utc=True important if dates are in diferent formats
accounting['creation_date'] = pd.to_datetime(accounting['creation_date'],utc=True)
accounting['execution_date'] = pd.to_datetime(accounting['execution_date'],utc=True)

In [282]:
def getimages(x):
    receipt = ''
    if not (x):
        return receipt
    elif(len(x)==0):
        return receipt
    elif('drive_url' in x[0].keys()):
        download_url = x[0]['drive_url']
        if(download_url != None):
            return download_url
    elif('download_url' in x[0].keys()):
        download_url = x[0]['download_url']
        if(download_url != None):
            return download_url
    else:
        return ''
    
    return receipt

In [245]:
accounting['receipt']=accounting.images.apply(lambda x: getimages(x))

In [283]:
def getimages_2(x):
    drive_url = ''
    download_url = ''
    if not (x):
        drive_url = ''
    elif(0 < len(x)):
        if('drive_url' in x[0].keys()):
            drive_url = x[0]['drive_url']
        if('download_url' in x[0].keys()):
            download_url = x[0]['download_url']
    
    return pd.Series([drive_url,download_url])
accounting[['drive_url','download_url']] = accounting.images.apply(lambda x: getimages_2(x))

##### double check

In [246]:
accounting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2403 entries, 0 to 2402
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   images          2398 non-null   object             
 1   creation_date   2403 non-null   datetime64[ns, UTC]
 2   user_in_charge  2403 non-null   object             
 3   code            2403 non-null   int64              
 4   account_id      2403 non-null   object             
 5   vat_amount      2403 non-null   float64            
 6   intervention    2403 non-null   object             
 7   project         2403 non-null   object             
 8   id              2403 non-null   object             
 9   concept         2403 non-null   object             
 10  target_id       2403 non-null   object             
 11  phase           800 non-null    object             
 12  amount          2403 non-null   float64            
 13  vat             2403 non-null   f

### select columns

In [247]:
#and the order whe want to upload to bigquery
df = accounting[['concept', 'place', 'creation_date', 'vat', 'description', 'amount',
       'id', 'phase', 'creator_user', 'execution_date',
       'user_in_charge', 'origin', 'account_id', 'project', 'code', 'type',
       'intervention', 'target_id', 'vat_amount', 'receipt']]

## upload to BIGQUERY

In [253]:
table_id = "{}.accounting".format(dataset.dataset_id)
proj = bgq_client.project
df.to_gbq(table_id,proj,if_exists='replace')

1it [00:06,  6.42s/it]


In [260]:
#25 de Enero... debido a la buena cagada
#table_id = "{}.accounting_202101".format(dataset.dataset_id)
#proj = bgq_client.project

In [261]:
#df.to_gbq(table_id,proj,if_exists='replace')

1it [00:05,  5.75s/it]


## CHECK

In [266]:
df_check = pd.read_gbq('SELECT * FROM `oan-nikarit.contoan.accounting`')

Downloading: 100%|██████████| 2404/2404 [00:01<00:00, 1374.88rows/s]


In [267]:
len(df_check['receipt'].unique())

341

In [265]:
df_check[df_check['id']=='itQROxDOKoHOykjk4g9t']['receipt'].iloc[0]

'https://drive.google.com/file/d/13Ayx4I0_GuFJQg5bmeHaYGPL0vrsOlFM/view'

### SUBIR VENTAS

#### get data

In [11]:
col_query = db.collection('Nikarit_Sales').stream()

In [12]:
ventas = []
for acc in col_query:
    ventas.append(acc.to_dict())

In [13]:
df_ventas = pd.DataFrame(ventas)

In [14]:
df_ventas.shape

(1775, 69)

### view

In [15]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1775 entries, 0 to 1774
Data columns (total 69 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   bolsa_nb                   513 non-null    float64
 1   packBandana_total_price    493 non-null    float64
 2   product_movement           458 non-null    object 
 3   navidadCode                403 non-null    float64
 4   packHidra_nb               1775 non-null   float64
 5   balsamo_unit_price         1775 non-null   object 
 6   gateway                    1775 non-null   object 
 7   packHidra_unit_price       1775 non-null   object 
 8   country                    1009 non-null   object 
 9   client_phone               429 non-null    object 
 10  manteca_nb                 1775 non-null   float64
 11  subtotal_price             1775 non-null   object 
 12  packMadre_unit_price       187 non-null    float64
 13  client_gender              1128 non-null   objec

#### transform data

In [16]:
cols = list(df_ventas.columns)

In [17]:
cambios = [col for col in cols if 'price' in col]

In [18]:
for cambio in cambios:
    df_ventas[cambio]=pd.to_numeric(df_ventas[cambio])

In [19]:
df_ventas['creation_date']=pd.to_datetime(df_ventas['creation_date'],utc=True)
df_ventas['close_date']=pd.to_datetime(df_ventas['close_date'],utc=True)

In [20]:
#double check
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1775 entries, 0 to 1774
Data columns (total 69 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   bolsa_nb                   513 non-null    float64            
 1   packBandana_total_price    493 non-null    float64            
 2   product_movement           458 non-null    object             
 3   navidadCode                403 non-null    float64            
 4   packHidra_nb               1775 non-null   float64            
 5   balsamo_unit_price         1775 non-null   float64            
 6   gateway                    1775 non-null   object             
 7   packHidra_unit_price       1775 non-null   float64            
 8   country                    1009 non-null   object             
 9   client_phone               429 non-null    object             
 10  manteca_nb                 1775 non-null   float64            
 11  subt

In [21]:
df_ventas.shape

(1775, 69)

In [22]:
df_ventas['toteBag_nb'] = 0
df_ventas['toteBag_unit_price'] = 0
df_ventas['toteBag_total_price'] = 0

### METER MUJERES Y HOMBRES

In [23]:
#client_gender

In [24]:
#lista de mujeres
url = 'https://raw.githubusercontent.com/marcboquet/spanish-names/master/mujeres.csv'
df_mujeres = pd.read_csv(url, error_bad_lines=False)
l_mujeres = df_mujeres['nombre'].to_list()

In [25]:
l_mujeres = [str(nom).lower() for nom in l_mujeres]

In [26]:
l_mujeres= l_mujeres+['maasuncion','mirenchu','begona','nadai','sra.','mcarmen','carmena','maita',
'madolores','reyero','ma','maconcepción','cris']

In [27]:
#lista de hombres
url = 'https://raw.githubusercontent.com/marcboquet/spanish-names/master/hombres.csv'
df_hombres = pd.read_csv(url, error_bad_lines=False)
l_hombres = df_hombres['nombre'].to_list()

In [28]:
l_hombres = [nom.lower() for nom in l_hombres]

In [29]:
l_hombres= l_hombres+['fcojavier']

In [30]:
#aplicar sexo
df_ventas['client_gender'] = df_ventas['client_name'].apply(lambda x: 
                            'F' if unidecode.unidecode(
                                    str(x).replace(" ", "").lower()) in l_mujeres
                            else 'M' if unidecode.unidecode(
                                str(x).replace(" ", "").lower())  in l_hombres
                            else 'F' if unidecode.unidecode(
                                str(x).split(" ")[0].lower())  in l_mujeres
                            else 'M' if unidecode.unidecode(
                                str(x).split(" ")[0].lower())  in l_hombres

                            else 'N/A')

In [31]:
#ver que sale
df_ventas['client_gender'].value_counts() # A MEJORRAR

F      1510
M       221
N/A      44
Name: client_gender, dtype: int64

In [32]:
df_ventas[df_ventas['client_gender'] == 'N/A'
         ]['client_name'].apply(lambda x: unidecode.unidecode(str(x).replace(" ", "").lower()))

101                                 chinwe
121                             herguedas,
122                       notariaserrano19
144                      impulsandolab,s.l
151                                     f.
198                         beaaccemmurcia
211                    herbolariosalamanca
212                                  patri
269                  indianoceanconsulting
426                                     f.
569                              pachamama
598                   companerosmastersofi
623                         farmacialaluna
628                                herrera
635                                 gandou
651                                   none
671                                  nanna
704                                  madre
729                      companerostrabajo
791                                  durga
834                                   none
843                                   none
863                                   none
875        

## METER DONACIONES

In [33]:
df_ventas['origin'].unique()

array(['Shopify', 'sumup', 'Contoan'], dtype=object)

In [34]:
def don(x):
    total_price = x['total_price']
    subtotal_price = x['subtotal_price']
    fecha = str(x['creation_date']).split(' ')[0]
    origin = x['origin']
    don = 0.0
    if '2020-11-01' < fecha and origin == "Shopify":
        if total_price < 40:
            don = total_price - subtotal_price - 5.50
        else:
            don = total_price - subtotal_price
            
    return don

df_ventas['donation'] = df_ventas.apply(lambda x: don(x),axis=1)

In [35]:
df_ventas['donation'] = df_ventas.apply(lambda x: don(x),axis=1)

In [36]:
df_ventas['donation'].sum()

790.68

## ERROR EN PROVINCE

In [37]:
import requests

In [38]:
## DEBERIAMOS exportarlo bien a nuestra bbdd
shop_url = os.environ['SHOPIFY_ACCESS_URL']
dates = ['2019-01-01T00:15:47-04:00','2020-01-01T00:15:47-04:00',
         '2020-05-01T00:15:47-04:00','2020-12-10T00:15:47-04:00', '2021-01-10T00:15:47-04:00']
l_orders = []

for date_min,date_max in zip(dates[:-1],dates[1:]):
    orders_url = shop_url+'/orders.json'
    r = requests.get(orders_url,
                 params={
                        'limit':250,
                        'status':'any',
                         'created_at_max':date_max,
                         'created_at_min':date_min
                        })
    orders = pd.DataFrame(r.json()['orders'])
    l_orders.append(orders)
df_orders = pd.concat(l_orders,axis=0).reset_index().drop_duplicates(subset=['id'])

In [39]:
df_orders['province_2'] = df_orders['billing_address'].apply(lambda x: x['province'])
df_orders['order_id'] = df_orders['id'].apply(str)
df_ventas['order_id'] = df_ventas['order_id'].apply(str)
df_ventas = pd.merge(df_ventas,df_orders[['order_id','province_2']],on='order_id',how='left')

df_ventas['province'] = df_ventas['province_2'] 
df_ventas = df_ventas.drop('province_2',axis=1)

In [40]:
cols = list(df_ventas.columns)
cols.pop(cols.index('to'))
cols.pop(cols.index('message'))
cols.pop(cols.index('navidadCode'))

'navidadCode'

In [41]:
df_ventas = df_ventas[cols]

In [42]:
df_ventas.shape

(1775, 69)

In [43]:
df_ventas['province'].value_counts().head()

Madrid        427
Barcelona      45
Valladolid     42
Valencia       30
Málaga         22
Name: province, dtype: int64

## ACTUALIZAR FIREBAE

In [44]:
#l_ship_det = ['ship_'+a for a in list(df_ord['shipping_address'][0].keys())]
#df_ord['ship_zip']=df_ord['ship_zip'].fillna(value='')
#def update_firebase(x):
#    order_id = x['order_id']
#    if order_id==None :
#        return True
#    else:
#        order_id = int(order_id)
#    ship_zip=df_ord[df_ord['id']==order_id]['ship_zip'].values[0]
#    fire_id=x['id']
#    #print(x['creation_date'])
#    #a=db.collection(u'Nikarit_Sales').document(x['id']).update({'zip':ship_zip})
##df_ventas[df_ventas['origin']!='Contoan'].apply(lambda x: update_firebase(x),axis=1)

## CORREGIR SUBTOTAL Y IVA

In [45]:
def iva(x):
    iva = x['total_tax']
    
    #Corregir ventas de contoan
    if x['origin'] == 'Contoan':
        iva = x['total_price']*(1-1/1.21)
    return iva

df_ventas["total_tax_2"] = df_ventas.apply(lambda x: iva(x),axis=1)
df_ventas["total_tax"] = df_ventas["total_tax_2"]
df_ventas = df_ventas.drop('total_tax_2',axis=1)

In [46]:
def subtotal(x):
    subtotal = x['subtotal_price']
    
    #Corregir ventas de contoan
    if x['origin'] == 'Contoan':
        subtotal = x['total_price']
    return subtotal

df_ventas["subtotal_price_2"] = df_ventas.apply(lambda x: subtotal(x),axis=1)
df_ventas["subtotal_price"] = df_ventas["subtotal_price_2"]
df_ventas = df_ventas.drop('subtotal_price_2',axis=1)

In [66]:
cols = list(df_ventas.columns)

In [67]:
cols.sort()

In [54]:
#subir a bigquery
table_id = "contoan.sales"

df_ventas[cols].to_gbq(table_id, project_id=bgq_client.project, if_exists='replace')

1it [00:05,  5.07s/it]


## TRANSFORM BOOLEAN

In [55]:
sql = """
CREATE OR REPLACE TABLE `oan-nikarit.contoan.sales` AS
SELECT
  * EXCEPT (buyer_accepts_marketing,taxes_included),
  CAST(buyer_accepts_marketing AS BOOL) AS buyer_accepts_marketing,
  CAST(taxes_included AS BOOL) AS taxes_included
FROM
  `oan-nikarit.contoan.sales`
"""

In [57]:
bgq_client.query(sql)

### COPY TO FIREBASE CLOUD FUNCTION

In [69]:
print_cols = cols.copy()

In [70]:
print_cols.remove("buyer_accepts_marketing")
print_cols.remove("taxes_included")

In [72]:
print_cols = print_cols + ["buyer_accepts_marketing", "taxes_included"]

In [79]:
df_ventas[print_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1775 entries, 0 to 1774
Data columns (total 69 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   accounting_item            473 non-null    object             
 1   balsamo_nb                 1775 non-null   float64            
 2   balsamo_total_price        1775 non-null   float64            
 3   balsamo_unit_price         1775 non-null   float64            
 4   bolsa_nb                   513 non-null    float64            
 5   bolsa_total_price          513 non-null    float64            
 6   bolsa_unit_price           513 non-null    float64            
 7   city                       432 non-null    object             
 8   client_adresse             429 non-null    object             
 9   client_email               1069 non-null   object             
 10  client_gender              1775 non-null   object             
 11  clie

## Cuentas

In [11]:
#get all the collection
col_query = db.collection('Accounts').stream()

In [12]:
#save it in a list of dictionaries
cuentas = []
for acc in col_query:
    cuentas.append(acc.to_dict())

In [13]:
#convert to dataframe
df_accounts = pd.DataFrame(cuentas)

In [14]:
df_cuentas = df_accounts[['id','name','number','type','user_id']].copy()

In [20]:
with open('fr.json', 'r') as f:
    trads = json.load(f)

In [21]:
df_cuentas['name_fr'] = df_cuentas['name'].apply(lambda x: trads[x])

In [22]:
df_cuentas[df_cuentas['name_fr'].isnull()]

,id,name,number,type,user_id,name_fr


In [23]:
table_id = "{}.accounts".format(dataset.dataset_id)

df_cuentas.to_gbq(table_id, project_id=bgq_client.project, if_exists='replace')

1it [00:03,  3.66s/it]


### PROYECTOS

In [26]:
#get all the collection
col_query = db.collection('Projects').stream()

In [27]:
#save it in a list of dictionaries
projects = []
for acc in col_query:
    projects.append(acc.to_dict())

In [28]:
#convert to dataframe
df_projects = pd.DataFrame(projects)

In [29]:
df_projectos=df_projects[['id','title','description','department','type','project_id','intervention_id']]

In [30]:
df_projectos.columns = ['id', 'name', 'description', 'department', 'type', 'project_id',
       'intervention_id']

In [32]:
table_id = "{}.projects".format(dataset.dataset_id)

df_projectos.to_gbq(table_id, project_id=bgq_client.project, if_exists='replace')

1it [00:02,  2.71s/it]


### USUARIOS

In [50]:
#get all the collection
col_query = db.collection('Users').stream()

In [51]:
#save it in a list of dictionaries
usuarios = []
for acc in col_query:
    usuarios.append(acc.to_dict())

In [52]:
#convert to dataframe
df_usuarios = pd.DataFrame(usuarios)

In [53]:
df_users = df_usuarios[['id','name','last_name','email']]

In [54]:
df_users_final = pd.merge(df_users,df_cuentas[['number','user_id']],left_on='id',right_on='user_id',how='left')

In [55]:
df_users = df_users_final[['id','name','last_name','email','number']]

In [56]:
df_users.columns = ['id','name','last_name','email','cuenta']

In [58]:
table_id = "{}.users".format(dataset.dataset_id)

pandas_gbq.to_gbq(df_users, table_id, project_id=client.project, if_exists='replace')

1it [00:06,  6.33s/it]


##### september 2020 backup

In [67]:
table_id = "{}.accounting_202009".format(dataset.dataset_id)

pandas_gbq.to_gbq(df, table_id, project_id=client.project, if_exists='replace')

1it [00:04,  4.48s/it]


##### Jan 2021 backup

In [351]:
sql = """
CREATE TABLE AS `oan-nikarit.contoan.accounting_20210110`
AS
SELECT *
FROM `oan-nikarit.contoan.accounting`
"""

In [352]:
client.query(sql)

### SUBIR SOCIOS

#### get data

In [14]:
col_query = db.collection('Partners').stream()

In [15]:
socios = []
for acc in col_query:
    socios.append(acc.to_dict())

In [16]:
df_socios = pd.DataFrame(socios)

In [17]:
df_socios.shape

(1311, 34)

In [19]:
print(df_socios.columns)

Index(['address', 'DNI', 'phone_number', 'university', 'email', 'donator',
       'yearsInNikki', 'isSubscribedToNewsletter', 'leaving_date',
       'mailChimp_id', 'registration_date', 'birth_date',
       'acceptsPrivacyPolitics', 'categories', 'status', 'acceptsVoluntarily',
       'contactPerson', 'projects', 'language', 'isDonator', 'location',
       'welcoming_pack', 'degree', 'thesis', 'id', 'occupation',
       'acceptsDutiesAsPartner', 'last_name', 'db_creation_date', 'department',
       'email_id', 'name', 'last_update_date', 'timeInNikki'],
      dtype='object')


In [22]:
df_socios[['latitude','longitude','country_code']
         ] = df_socios['location'].apply(lambda x: pd.Series([x['lat'],x['lon'],x['country_code']]))

In [32]:
def donor_values(x,value,nonevalue):
    v = nonevalue
    if x['donator']:
        if value in x['donator'].keys():
            v = x['donator'][value]
    return v

In [33]:
df_socios['donator_periodic_amount'
         ] = df_socios.apply(lambda x: donor_values(x,'periodic_amount',0),axis=1)

In [34]:
df_socios['donator_period'
         ] = df_socios.apply(lambda x: donor_values(x,'period',''),axis=1)

In [35]:
df_socios['donator_start_date'
         ] = df_socios.apply(lambda x: donor_values(x,'start_date',''),axis=1)

In [36]:
df_socios['donator_total_amount'
         ] = df_socios.apply(lambda x: donor_values(x,'total_amount',0),axis=1)

In [37]:
df_socios_gbq = df_socios[
                ['db_creation_date', 
                 'registration_date',
                 'leaving_date',
                 'mailChimp_id',
                 'email',
                 'email_id',
                 'name',
                 'last_name',
                 'id',
                 'language',
                 'phone_number',
                 'status',
                 'DNI',
                 'birth_date',
                 'contactPerson',
                 'address',
                 'occupation',
                 'university',
                 'degree',
                 'latitude',
                 'longitude',
                 'country_code',
                 'categories',
                 'department',
                 'projects',
                 'thesis',
                 'isSubscribedToNewsletter',
                 'welcoming_pack',
                 'isDonator',
                 'donator_periodic_amount',
                 'donator_period',
                 'donator_start_date',
                 'donator_total_amount',
                 'acceptsVoluntarily',
                 'acceptsDutiesAsPartner',
                 'acceptsPrivacyPolitics']
                ]

In [65]:
df_socios_gbq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   db_creation_date          1311 non-null   object 
 1   registration_date         1311 non-null   object 
 2   leaving_date              0 non-null      object 
 3   mailChimp_id              1269 non-null   object 
 4   email                     1311 non-null   object 
 5   email_id                  1269 non-null   object 
 6   name                      1311 non-null   object 
 7   last_name                 1311 non-null   object 
 8   id                        1311 non-null   object 
 9   language                  1269 non-null   object 
 10  phone_number              288 non-null    object 
 11  status                    1311 non-null   object 
 12  DNI                       68 non-null     object 
 13  birth_date                137 non-null    object 
 14  contactP

In [67]:
dates_col = [col for col in df_socios_gbq.columns if 'date' in col]
print(dates_col)

['db_creation_date', 'registration_date', 'leaving_date', 'birth_date', 'donator_start_date']


In [70]:
for col in dates_col:
    df_socios_gbq[col] = pd.to_datetime(df_socios_gbq[col],utc=True)

<ipython-input-70-d9513b8394e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_socios_gbq[col] = pd.to_datetime(df_socios_gbq[col],utc=True)


In [71]:
df_socios_gbq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   db_creation_date          1311 non-null   datetime64[ns, UTC]
 1   registration_date         1260 non-null   datetime64[ns, UTC]
 2   leaving_date              0 non-null      datetime64[ns, UTC]
 3   mailChimp_id              1269 non-null   object             
 4   email                     1311 non-null   object             
 5   email_id                  1269 non-null   object             
 6   name                      1311 non-null   object             
 7   last_name                 1311 non-null   object             
 8   id                        1311 non-null   object             
 9   language                  1269 non-null   object             
 10  phone_number              288 non-null    object             
 11  status           

In [72]:
table_id = "{}.partners".format(dataset.dataset_id)
proj = bgq_client.project
df_socios_gbq.to_gbq(table_id,proj,if_exists='replace')

1it [00:04,  4.89s/it]


## TRANSFORM BOOLEAN

In [73]:
bools = ['acceptsVoluntarily','acceptsDutiesAsPartner','acceptsPrivacyPolitics',
         'isDonator','isSubscribedToNewsletter','welcoming_pack']

In [74]:
cast = ['CAST('+a+' AS BOOL) AS '+a for a in bools]

In [75]:
sql = """
CREATE OR REPLACE TABLE `oan-nikarit.contoan.partners` AS
SELECT
  * EXCEPT ("""+','.join(bools)+"""),
 """+',\n '.join(cast)+"""
FROM
  `oan-nikarit.contoan.partners`
"""

In [76]:
bgq_client.query(sql)

### SOCIOS

In [54]:
sheet = sheet_service.spreadsheets()

In [55]:
SPREADSHEET_ID = "1npTnlYkICz76ctoupzuEH-k1CbXoRh07czTgZDJLOtQ"

In [58]:
rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range = "libro de socios actualizado (no tocar)").execute()

In [62]:
df_socios = pd.DataFrame(rows['values'][2:],columns=rows['values'][2])

In [63]:
df_socios

,Nombre / Prénom,Apellidos / Nom,Email,Fecha de Nacimiento / Date de Naissance,Tipo de Estudiante / Tipe d'Étudiante,(España) Selecciona tu Universidad,(En caso afirmativo) Profesión,DNI / Nombre d'identité,Número de Teléfono / Numéro de Portable,Domicilio / Adresse,...,Departamento en el que colaboras,Declaro que la información anterior es correcta*,¿En qué momento has estado en Nikki?*,¿Durante Cuánto Tiempo (en total)?*,Quieres apoyar a OAN con una donación*,"En caso afirmativo, selecciona*",Acepto voluntariamente ser Socio de OAN*,Declaro cumplir con mis deberes como socio*,Fecha de registro,Fecha de último cambio
0,Nombre / Prénom,Apellidos / Nom,Email,Fecha de Nacimiento / Date de Naissance,Tipo de Estudiante / Tipe d'Étudiante,(España) Selecciona tu Universidad,(En caso afirmativo) Profesión,DNI / Nombre d'identité,Número de Teléfono / Numéro de Portable,Domicilio / Adresse,...,Departamento en el que colaboras,Declaro que la información anterior es correcta*,¿En qué momento has estado en Nikki?*,¿Durante Cuánto Tiempo (en total)?*,Quieres apoyar a OAN con una donación*,"En caso afirmativo, selecciona*",Acepto voluntariamente ser Socio de OAN*,Declaro cumplir con mis deberes como socio*,Fecha de registro,Fecha de último cambio
1,Rodrigo,Pacheco Martinez-Atienza,rpacheco95@gmail.com,1995-03-16,Grado / Licence,Universidad Rey Juan Carlos,,50482811L,651623729,"{addr1=C/ Reina 26 D, country=ES, zip=28300, s...",...,,Si / Oui,OAN2017,2 meses / 2 mois,,,Si / Oui,Si / Oui,2018-09-23,2020-05-12
2,Álvaro,Pascual Macías,alvaropsc@gmail.com,1994-02-02,Graduado / Diplômé,Universidad Complutense de Madrid,Médico,02309963G,695517703,"{addr2=, country=ES, city=Madrid, addr1=Calle ...",...,,Si / Oui,OAN2014,5 meses / 5 mois,No / Non,,Si / Oui,Si / Oui,2018-09-17,2020-05-12
3,Pablo,Jordán de la Fuente,Pablo.jordan.dlf@gmail.com,1994-07-08,Graduado / Diplômé,Universidad de Valladolid,Consultor,71167986Y,660087618,"{addr2=, zip=28003, city=Madrid, addr1=Calle A...",...,Departamento de Proyectos / Projets,Si / Oui,OAN2016,1 mes / 1 mois,,,Si / Oui,Si / Oui,2018-12-02,2020-05-12
4,ESTHER,FRAILE JULIAN,Estherfrailejulian@gmail.com,1993-10-16,Master / Master,Universidad Autonoma de Madrid,PSICOLOGA,05291047N,665584356,"{addr1=CALLE LOPEZ MORENO 12, 3D, city=MELILLA...",...,Departamento de Proyectos / Projets,Si / Oui,OAN2017,más 6 meses / Plus de 6 mois,No / Non,,Si / Oui,Si / Oui,2018-09-17,2020-05-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Antonio Manuel,Marín Fernández,antoniomarin200294@gmail.com,1994-02-20,Estudiante (Grado) / Étudiant,Universidad Pontificia Comillas,DERECHO,48664434E,692356132,CALLE FERNÁNDEZ DE LOS RÍOS Nº77,...,Ninguno / Aucun,,No,,"No, gracias / Non, merci",,Sí / Oui,Sí / Oui,2020-10-12,None
65,Rolando,Alem,r.alem@oaninternational.org,1989-03-05,Estudiante (Grado) / Étudiant,Universidad Rey Juan Carlos,Ingenieria en tecnologias industriales,Y521394D,659210762,Calle de la reina 25,...,"Comité de Energía, Agua e Infraestructuras",,No,,"No, gracias / Non, merci",,Sí / Oui,Sí / Oui,2021-01-23,None
66,Ignacio,Gómez-Cambronero Gutiérrez de Cabiedes,ignaciogcgc@alu.comillas.edu,1997-10-16,Estudiante (Grado) / Étudiant,Universidad Pontificia Comillas,Grado en Ingeniería de Tecnologías Industriales,54361871Z,680461734,C/Solano 18,...,"Comité de Energía, Agua e Infraestructuras",,No,,"No, gracias / Non, merci",,Sí / Oui,Sí / Oui,2021-01-25,None
67,Sandra,Cózar Carbonell,s.cozar@oaninternational.org,1992-11-20,Profesional / Professionnel,Universidad Pablo de Olavide,Doble Grado en Derecho y Ciencias Políticas de...,05937108A,618478186,C/Niceto Alcalá Zamora,...,Comité de Políticas Sociales,,No,,"No, gracias / Non, merci",,Sí / Oui,Sí / Oui,2021-01-26,None
